In [1]:
import cPickle, gzip, numpy

## Load the dataset
f = gzip.open('./mnist.pkl.gz', 'rb')
train_set, valid_set, test_set = cPickle.load(f)
f.close()


In [15]:
from sklearn.svm import LinearSVC
from sklearn.calibration import CalibratedClassifierCV

print "train_setSize= %s, train_set_labels= %s, %s" \
%(train_set[0].shape, train_set[1].shape,  type(train_set[0]))

## The cPickle method returns tuple with training cases and labels. 
## We need to extract them form these

X_train = train_set[0][:]
Y_train = train_set[1][:]


print ("X_train= %s, %s\nY_train= %s, %s\n") \
%(X_train.shape, type(X_train), Y_train.shape, type(Y_train))


svm = LinearSVC()
clf = CalibratedClassifierCV(svm)

## Training the classifier
## Takes about 10-15 min to train
clf.fit(X_train, Y_train)

train_setSize= (50000, 784), train_set_labels= (50000,), <type 'numpy.ndarray'>
X_train= (50000, 784), <type 'numpy.ndarray'>
Y_train= (50000,), <type 'numpy.ndarray'>



CalibratedClassifierCV(base_estimator=LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0),
            cv=3, method='sigmoid')

In [ ]:
## This loop was for polynomial SVM
# from sklearn import svm


# print "train_setSize= %s, train_set_labels= %s, %s" \
# %(train_set[0].shape, train_set[1].shape,  type(train_set[0]))

# ## The cPickle method returns tuple with training cases and labels. 
# ## We need to extract them form these

# X_train = train_set[0][:]
# Y_train = train_set[1][:]


# print ("X_train= %s, %s\nY_train= %s, %s\n") \
# %(X_train.shape, type(X_train), Y_train.shape, type(Y_train))

# clf = svm.SVC(probability=True, gamma=0.07)

# ## Training the classifier
# ## Takes about 10-15 min to train
# clf.fit(X_train, Y_train)




train_setSize= (50000, 784), train_set_labels= (50000,), <type 'numpy.ndarray'>
X_train= (50000, 784), <type 'numpy.ndarray'>
Y_train= (50000,), <type 'numpy.ndarray'>



In [16]:
## Validating the classifier
X_valid = valid_set[0]
Y_valid = valid_set[1]

print "X_valid= %s, %s\nY_valid= %s, %s\n" \
%(X_valid.shape, type(X_valid), Y_valid.shape, type(Y_valid))

## Score of the trained data set on the validation set
score = clf.score(X_valid, Y_valid)

print "Score on Valid set= %f" %(score)


X_valid= (10000, 784), <type 'numpy.ndarray'>
Y_valid= (10000,), <type 'numpy.ndarray'>

Score on Valid set= 0.921200


In [17]:
## get score on the test data
X_test = test_set[0]
Y_test = test_set[1]

print "X_test= %s, %s\nY_test= %s, %s\n"\
%(X_test.shape, type(X_test), Y_test.shape, type(Y_test) )

scoreTest = clf.score(X_test, Y_test)

print "Score on Test data= %f\n" %(scoreTest)


X_test= (10000, 784), <type 'numpy.ndarray'>
Y_test= (10000,), <type 'numpy.ndarray'>

Score on Test data= 0.916700



In [35]:
## getting probabilities for each test data
## Creates the probabilities for each class according to the labels.

probTest = clf.predict_proba(X_test)[:]


print "probTest = %s, shape= %s\n" %(probTest, probTest.shape)

print "probTest[0]= %s, len= %d\n" \
%(probTest[0], len(probTest[0]))

## index
index= 0
## Creates the predicted class for each testing value
yPredictTest = clf.predict(X_test)[:]

## max(probTest[index]) gives the max value for the array of predicted class. 
## This is the predicted score which is taken for classification
print "Predicted Class= %d, True Class= %d, Predicted Score= %f" \
%(yPredictTest[index], Y_test[index], max(probTest[index]) )

print "Class\tProbability"
for i in range(len(probTest[index])):
    print "%d\t%f" %(i, probTest[index][i])


## using str.format
print "Prob= {0:.6f}\n".format(probTest[index][0])

print "Str= %s\n" %(str('{0:.6f}'.format(probTest[index][0])))

## Open a csv for writing
with open('LinearSVM.csv', 'w') as f:
    f.write('True Class, Assigned Class, Correct, Predicted Score, C0, C1, C2, C3, C4, C5, C6, C7, C8, C9\n')
    
    for j in range(len(Y_test)):
#         trueClass = Y_test[j]
#         assignedClass = yPredictTest[j]
#         predictedScore = max(probTest[j])
        
        if (Y_test[j] == yPredictTest[j]) :
            correct = 1
        else:
            correct = 0
        
        f.write(str(Y_test[j]) + ", " + str(yPredictTest[j]) \
                + ", " + str(correct) + ", " + str('{0:.6f}'.format(max(probTest[j])) ) + ", " )
        
        for k in range(len(probTest[j])):
            f.write(str('{0:.6f}'.format(probTest[j][k]) ) + ", ")
        
        f.write("\n")
            


probTest = [[  1.01755709e-04   2.63131116e-11   3.35260610e-04 ...,   9.47836952e-01
    1.32619344e-04   1.49173139e-03]
 [  2.55525166e-03   6.31398567e-06   9.63656024e-01 ...,   1.37888506e-14
    1.59547670e-05   3.33174704e-11]
 [  1.39735901e-05   9.20197009e-01   3.13391685e-02 ...,   3.48832729e-03
    8.12310436e-03   3.96072140e-03]
 ..., 
 [  1.32779921e-07   2.39715019e-08   2.95231465e-05 ...,   7.97462077e-03
    7.38856967e-02   6.36063969e-02]
 [  7.14460813e-07   8.03578027e-07   2.83348744e-05 ...,   2.11268021e-05
    2.36234892e-01   1.21452253e-06]
 [  6.67587323e-06   3.24328225e-13   1.16609499e-03 ...,   1.23337716e-09
    3.53493542e-06   5.68031916e-07]], shape= (10000, 10)

probTest[0]= [  1.01755709e-04   2.63131116e-11   3.35260610e-04   4.97038598e-02
   8.83397963e-06   3.88944760e-04   4.24878026e-08   9.47836952e-01
   1.32619344e-04   1.49173139e-03], len= 10

Predicted Class= 7, True Class= 7, Predicted Score= 0.947837
Class	Probability
0	0.000102
1